# 04-5_evaluation_execution.ipynb — モジュール5 最終評価実行

- 目的: 偽陰性データに対するAIエージェント評価の実行、統計分析、レポート生成
- 入力: `artifacts/{RUN_ID}/handoff/04-4_agent_implementation.pkl`
- 出力:
  - `artifacts/{RUN_ID}/handoff/04-5_evaluation_results.pkl`
  - `artifacts/{RUN_ID}/results/evaluation_results_{timestamp}.csv`
  - `artifacts/{RUN_ID}/results/performance_metrics_{timestamp}.json`
  - `artifacts/{RUN_ID}/results/error_analysis_{timestamp}.json`

> 本ノートブックは `03_ai_agent_analysis_part4_config_patched_structured_output_v2_FIXED3.ipynb`
> から以下のセルを移植しています: 07-SIMPLE, 07-RERUN-FIX, 08-INVESTIGATE, 25(Controller API)。


In [1]:
# === Cell 0 (02以降 共通): レジストリから解決して paths を読む ===
try:
    import run_id_registry as runreg
    rid = runreg.bootstrap()  # env→ファイル→latest→新規
    import importlib
    import _compat.paths as paths
    importlib.reload(paths)
    print("[NX] RUN_ID =", rid, "| paths.RUN_ID =", getattr(paths, "RUN_ID", None))
except Exception as e:
    print("[04-5] registry/paths not available -> skipping. Reason:", e)
    rid = None


[NX] RUN_ID = 2025-10-22_074213 | paths.RUN_ID = 2025-10-22_074213


In [2]:
# === COMPAT shim: load_configuration (inject) ===
# - Provides load_configuration(cfg_override) expected by generalization_study()
# - Tolerates absence of _compat.paths by falling back to local config.json
from typing import Optional, Dict, Any
from pathlib import Path
import json, sys, os

try:
    import _compat.paths as _px
except Exception:
    _px = None  # allow fallback

def _deep_update(base: Dict[str, Any], upd: Optional[Dict[str, Any]]) -> Dict[str, Any]:
    if not isinstance(base, dict):
        return upd or {}
    if not isinstance(upd, dict):
        return base
    out = dict(base)
    for k, v in upd.items():
        if isinstance(v, dict) and isinstance(out.get(k), dict):
            out[k] = _deep_update(out[k], v)
        else:
            out[k] = v
    return out

def load_configuration(cfg_override: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    # 1) Try compat layer if available
    try:
        if _px and hasattr(_px, "ensure_roots"):
            try:
                _px.ensure_roots()
            except Exception:
                pass
        base = {}
        if _px and hasattr(_px, "load_config"):
            try:
                base = _px.load_config()
            except Exception:
                base = {}
        # 2) Fallback: local config.json candidates
        if not base:
            candidates = [
                Path("/mnt/data/config.json"),
                Path("config.json"),
                Path("_compat/config.json"),
            ]
            for p in candidates:
                if p.exists():
                    base = json.loads(p.read_text(encoding="utf-8"))
                    break
        if not base:
            raise FileNotFoundError("Missing config.json (tried /mnt/data, cwd, _compat).")
        # 3) Merge overrides
        return _deep_update(dict(base), cfg_override or {})
    except Exception as e:
        raise


In [3]:

# Cell 00: 初期化（Paths/Config bootstrap）
# - RUN_IDの決定と成果物ディレクトリの統一化
# - config.jsonのロード＆オーバーライド（DEV/PROD）
# - 乱数種の固定

import os, json, sys, time, pickle
from pathlib import Path
from datetime import datetime
import random
import numpy as np
import pandas as pd

# 互換パスの読込（artifacts/{RUN_ID}/... へ統一）
# /mnt/data/paths.py は drop-in shim として利用
paths_module = None
try:
    sys.path.append("_compat/")
    import paths as _compat_paths  # drop-in shim
    paths_module = _compat_paths
    _compat_paths.ensure_roots()
    RUN_ID = _compat_paths.load_config().get("run_id")
    BASE_DIRS = _compat_paths.compat_base_dirs
except Exception as e:
    print("[WARN] paths shimの初期化に失敗しました:", e)
    # フォールバック: カレント直下に artifacts/{RUN_ID}/.. を構築
    RUN_ID = os.environ.get("RUN_ID") or datetime.now().strftime("%Y-%m-%d_%H%M%S")
    ARTIFACTS = Path("artifacts") / RUN_ID
    for sub in ["raw","processed","models","results","handoff","logs","traces"]:
        (ARTIFACTS / sub).mkdir(parents=True, exist_ok=True)
    BASE_DIRS = {k: str(ARTIFACTS / v) for k, v in {
        'raw': 'raw','data':'processed','models':'models','results':'results',
        'handoff':'handoff','logs':'logs','traces':'traces'
    }.items()}

# Config読み込み
DEFAULT_CONFIG_PATH = Path("_compat/config.json")
cfg = {
    "system": {"development_mode": False, "seed": 42},
    "engine": {},
    "paths": {"results_dir": "results"}
}
if DEFAULT_CONFIG_PATH.exists():
    with open(DEFAULT_CONFIG_PATH, "r", encoding="utf-8") as f:
        cfg_from_file = json.load(f)
        # Shallow merge (keep structure simple for robustness)
        for k,v in cfg_from_file.items():
            if isinstance(v, dict) and k in cfg and isinstance(cfg[k], dict):
                cfg[k].update(v)
            else:
                cfg[k] = v

# 環境変数/引数によるDEVモード上書き
ENV_DEV = os.environ.get("DEV_MODE")
if ENV_DEV is not None:
    cfg["system"]["development_mode"] = (ENV_DEV.lower() == "true")

# 乱数seed
SEED = int(cfg.get("system",{}).get("seed", 42))
random.seed(SEED); np.random.seed(SEED)

print("RUN_ID:", RUN_ID)
print("development_mode:", cfg["system"]["development_mode"])
print("results_dir:", BASE_DIRS.get("results"))
print("handoff_dir:", BASE_DIRS.get("handoff"))


RUN_ID: 2025-10-22_074213
development_mode: False
results_dir: artifacts/2025-10-22_074213/results
handoff_dir: artifacts/2025-10-22_074213/handoff


In [4]:

# Cell 01: 入力ハンドオフの読込とモード別パラメータ
from pathlib import Path
import pickle, json
from datetime import datetime

handoff_path = Path(BASE_DIRS["handoff"]) / "04-4_agent_implementation.pkl"
print("Expect handoff:", handoff_path)

handoff = None
if handoff_path.exists():
    try:
        with open(handoff_path, "rb") as f:
            handoff = pickle.load(f)
        print("[OK] handoff loaded.")
    except Exception as e:
        print("[ERROR] handoff load failed:", e)
else:
    print("[WARN] handoff file not found (NOT_FOUND). 処理は実行時にフォールバックします。")

# 実行モード別の標準パラメータ
if cfg["system"]["development_mode"]:
    SAMPLES = 50
    MAX_WORKERS = 12
    SAMPLING = "low-ml-priority"  # 低ML確率優先
else:
    SAMPLES = 200
    MAX_WORKERS = 10
    SAMPLING = "random"
print({"SAMPLES": SAMPLES, "MAX_WORKERS": MAX_WORKERS, "SAMPLING": SAMPLING})


Expect handoff: artifacts/2025-10-22_074213/handoff/04-4_agent_implementation.pkl
[OK] handoff loaded.
{'SAMPLES': 200, 'MAX_WORKERS': 10, 'SAMPLING': 'random'}


In [5]:
# Cell 01.1: Handoff ローダ（堅牢化 + フォールバック）
# - 空/壊れた pkl("Ran out of input") を検知
# - artifacts/*/handoff を走査し、直近の有効 04-4 handoff を自動採用
# - 必須キーを検証
# - "thin handoff"（agent_spec→再構築）にも対応
from __future__ import annotations

from pathlib import Path
import pickle, json, os, glob, time
from typing import Dict, Any, Optional, Tuple

REQUIRED_KEYS = [
    "cfg", "HIGH_RISK_WORDS", "DANGEROUS_TLDS", "cert_full_info_map", "brand_keywords"
]

def _safe_pickle_load(p: Path) -> Tuple[Optional[dict], Optional[str]]:
    """戻り値: (obj or None, err or None)"""
    if not p.exists():
        return None, "NOT_FOUND"
    try:
        st = p.stat()
    except Exception as e:
        return None, f"STAT_ERROR: {e!r}"
    if st.st_size == 0:
        return None, "EMPTY_FILE"
    try:
        with p.open("rb") as f:
            obj = pickle.load(f)
    except EOFError:
        return None, "EOF (Ran out of input)"
    except Exception as e:
        return None, f"UNPICKLE_ERROR: {e!r}"
    if not isinstance(obj, dict):
        # 互換: (ReturnCode, PathsMap) などを許容
        if isinstance(obj, tuple) and len(obj) == 2 and isinstance(obj[1], dict):
            obj = {"return_code": obj[0], "paths": obj[1]}
        else:
            # 想定外だが dict に変換（出来る範囲で）
            try:
                obj = dict(obj)
            except Exception:
                return None, f"INVALID_OBJECT_TYPE: {type(obj)}"
    return obj, None

def _iter_candidate_paths() -> list[Path]:
    # まずは 04-5で期待している handoff_path を優先
    primaries = []
    try:
        primaries.append(handoff_path)  # defined in Cell 01
    except NameError:
        pass

    # artifacts/*/handoff/04-4_agent_implementation.pkl を新しい順に
    roots = [Path("artifacts"), Path("/mnt/data/artifacts")]
    cands = []
    for root in roots:
        if root.exists():
            for p in root.glob("*/handoff/04-4_agent_implementation.pkl"):
                try:
                    cands.append((p.stat().st_mtime, p))
                except Exception:
                    continue
    cands.sort(key=lambda t: t[0], reverse=True)
    ordered = list(dict.fromkeys(primaries + [p for _, p in cands]))  # 重複除去し順序保持
    return ordered

def _maybe_reconstruct_from_agent_spec(h: dict) -> dict:
    # agent_spec が dict で保存されている場合、必要なら再構築のフックを用意
    spec = h.get("agent_spec")
    if spec and isinstance(spec, dict):
        try:
            # ここでは重い再構築は行わず、必要なキーが無ければ WARN のみ
            pass
        except Exception as e:
            print("[WARN] agent_spec 再構築に失敗:", e)
    return h

def _validate_required_keys(h: dict) -> list[str]:
    return [k for k in REQUIRED_KEYS if k not in h]

# 実体ロード
handoff_ok = False
handoff_src = None
_handoff_errors = []

for p in _iter_candidate_paths():
    obj, err = _safe_pickle_load(p)
    if err:
        _handoff_errors.append((str(p), err))
        continue
    # agent_spec best-effort
    obj = _maybe_reconstruct_from_agent_spec(obj)
    miss = _validate_required_keys(obj)
    if miss:
        print("[WARN] handoff missing keys:", miss, "at", p)
        # 必須キー欠損でもフォールバック候補としては保持し、次の候補を試す
        # 最終的に一番新しい成功候補を採用
    # 候補採用（最初に見つかった正常ファイルで良い）
    handoff = obj
    handoff_path = p
    handoff_ok = True
    handoff_src = "primary" if ('primaries' in locals() and primaries and p == primaries[0]) else "fallback"
    break

if not handoff_ok:
    # 最後の試行: /mnt/data 直下の雑多な場所を軽くスキャン（安全のため深くは見ない）
    misc_globs = list(Path("/mnt/data").glob("**/04-4_agent_implementation.pkl"))
    misc_globs = [p for p in misc_globs if "/.ipynb_checkpoints/" not in str(p)]
    misc_globs.sort(key=lambda p: p.stat().st_mtime if p.exists() else 0, reverse=True)
    for p in misc_globs[:10]:
        obj, err = _safe_pickle_load(p)
        if err:
            _handoff_errors.append((str(p), err))
            continue
        handoff = _maybe_reconstruct_from_agent_spec(obj)
        handoff_path = p
        handoff_ok = True
        handoff_src = "fallback-misc"
        break

# ログと検証
if handoff_ok:
    print(f"[OK] handoff ready: {handoff_path} (source={handoff_src})")
    missing = _validate_required_keys(handoff)
    if missing:
        print("[WARN] handoff is missing keys:", missing)
else:
    print("[ERROR] handoff ロード失敗履歴:")
    for p, err in _handoff_errors[:10]:
        print("  -", p, "->", err)
    print("[FATAL] No valid 04-4 handoff found. Please run Module 04-4 to generate it.")


[WARN] handoff missing keys: ['HIGH_RISK_WORDS', 'DANGEROUS_TLDS', 'cert_full_info_map', 'brand_keywords'] at artifacts/2025-10-22_074213/handoff/04-4_agent_implementation.pkl
[OK] handoff ready: artifacts/2025-10-22_074213/handoff/04-4_agent_implementation.pkl (source=fallback)
[WARN] handoff is missing keys: ['HIGH_RISK_WORDS', 'DANGEROUS_TLDS', 'cert_full_info_map', 'brand_keywords']


In [6]:
"""
プログラム名: 03_ai_agent_analysis.ipynb
概要: MLモデルが見逃したフィッシングサイトをAIエージェント（Qwen3-14B）で検出
目的: 国際学会論文での発表（MLとLLMのハイブリッドアプローチの実証）
作成日: 2025-01-17
"""

"""
セル番号: 07-SIMPLE
概要: シンプルな動作確認版（まず5件でテスト）
入力: false_negatives_df, evaluate_domain
出力: テスト結果の表示と動作確認
"""

import time
import pandas as pd
from datetime import datetime

print("=" * 80)
print("🧪 シンプル動作テスト（5件のみ）")
print("=" * 80)

# ========== 1. vLLMサーバー接続確認 ==========

print("\n1️⃣ vLLMサーバー接続確認:")
print("-" * 40)

try:
    from langchain_openai import ChatOpenAI
    
    # vLLM設定の確認
    if 'VLLM_CONFIG' in globals():
        print(f"✅ VLLM設定:")
        print(f"   - URL: {VLLM_CONFIG['base_url']}")
        print(f"   - Model: {VLLM_CONFIG['model']}")
        
        # 簡単なテスト
        test_llm = ChatOpenAI(**VLLM_CONFIG)
        response = test_llm.invoke("Say 'OK' if you are working")
        print(f"✅ vLLMサーバー応答確認")
        print(f"   サーバー側でGPUが使用されています")
    else:
        print("❌ VLLM_CONFIG が未定義")
        print("   セル06を実行してください")
        
except Exception as e:
    print(f"❌ vLLMサーバー接続エラー: {e}")

# ========== 2. 評価関数の確認 ==========

print("\n2️⃣ 評価関数の確認:")
print("-" * 40)

if 'evaluate_domain' in globals():
    print("✅ evaluate_domain 関数が定義されています")
else:
    print("❌ evaluate_domain が未定義です")
    print("   セル06を実行してください")

# ========== 3. テストデータの準備 ==========

print("\n3️⃣ テストデータ準備:")
print("-" * 40)

if 'false_negatives_df' not in globals():
    print("❌ false_negatives_df が未定義です")
    print("   セル01または02でデータを読み込んでください")
else:
    # 5件だけ取得
    test_samples = false_negatives_df.head(5).copy()
    print(f"✅ テストサンプル: {len(test_samples)}件")
    
    # ML確率カラムの確認
    prob_col = 'prediction_proba' if 'prediction_proba' in test_samples.columns else 'ml_probability'
    
    print("\n📋 テストドメイン:")
    for idx, row in test_samples.iterrows():
        print(f"   {idx+1}. {row['domain'][:50]:50s} (ML: {row[prob_col]:.3f})")

# ========== 4. 1件ずつ評価（最もシンプル） ==========

print("\n4️⃣ 評価実行（1件ずつ処理）:")
print("-" * 40)

def simple_evaluate_one(domain, ml_prob):
    """1件だけ評価する最もシンプルな関数"""
    
    print(f"\n評価中: {domain}")
    print(f"  ML確率: {ml_prob:.3f}")
    
    start = time.time()
    
    try:
        # evaluate_domain を呼び出し
        result = evaluate_domain(domain, ml_prob, enable_trace=False)
        
        elapsed = time.time() - start
        
        # 結果の表示
        is_phishing = result.get('ai_is_phishing', False)
        confidence = result.get('ai_confidence', 0.0)
        
        print(f"  結果: {'🚨 フィッシング' if is_phishing else '✅ 正常'}")
        print(f"  信頼度: {confidence:.2f}")
        print(f"  処理時間: {elapsed:.1f}秒")
        
        # 詳細情報
        if result.get('detected_brands'):
            print(f"  検出ブランド: {', '.join(result['detected_brands'])}")
        if result.get('risk_factors'):
            print(f"  リスク要因: {', '.join(result['risk_factors'][:3])}")
        
        return result
        
    except Exception as e:
        print(f"  ❌ エラー: {e}")
        return {
            'domain': domain,
            'ml_probability': ml_prob,
            'ai_is_phishing': False,
            'ai_confidence': 0.0,
            'error': str(e)
        }

# ========== 5. テスト実行 ==========

if 'test_samples' in locals() and 'evaluate_domain' in globals():
    print("\n🚀 テスト評価を開始します...")
    print("=" * 80)
    
    results = []
    total_start = time.time()
    
    # 1件ずつ処理
    for idx, row in test_samples.iterrows():
        domain = row['domain']
        ml_prob = row[prob_col]
        
        result = simple_evaluate_one(domain, ml_prob)
        results.append(result)
        
        # 少し待機（サーバー負荷軽減）
        time.sleep(0.5)
    
    total_elapsed = time.time() - total_start
    
    # ========== 6. 結果サマリー ==========
    
    print("\n" + "=" * 80)
    print("📊 テスト結果サマリー")
    print("=" * 80)
    
    # 統計
    detected = sum(1 for r in results if r.get('ai_is_phishing', False))
    errors = sum(1 for r in results if r.get('error'))
    
    print(f"\n統計:")
    print(f"  - 処理件数: {len(results)}件")
    print(f"  - フィッシング検出: {detected}件 ({detected/len(results)*100:.1f}%)")
    print(f"  - エラー: {errors}件")
    print(f"  - 総処理時間: {total_elapsed:.1f}秒")
    print(f"  - 平均処理時間: {total_elapsed/len(results):.1f}秒/件")
    
    # 結果詳細
    print(f"\n詳細結果:")
    for i, result in enumerate(results, 1):
        domain = test_samples.iloc[i-1]['domain']
        is_phishing = "🚨 フィッシング" if result.get('ai_is_phishing') else "✅ 正常"
        confidence = result.get('ai_confidence', 0.0)
        print(f"  {i}. {domain[:40]:40s} → {is_phishing} (信頼度: {confidence:.2f})")
    
    # DataFrameに変換
    results_df = pd.DataFrame(results)
    
    print("\n" + "=" * 80)
    print("✅ テスト完了！")
    print("=" * 80)
    
else:
    print("\n❌ テストを実行できません")
    print("   必要な変数または関数が未定義です")

# ========== 7. 次のステップ ==========

print("\n💡 次のステップ:")
print("-" * 40)

if 'results' in locals() and len(results) > 0:
    avg_time = total_elapsed / len(results)
    estimated_total = 4215 * avg_time / 60
    
    print(f"✅ テスト成功！")
    print(f"   平均処理時間: {avg_time:.1f}秒/件")
    print(f"   4,215件の推定処理時間: {estimated_total:.1f}分")
    
    print("\n全件処理を開始する場合:")
    print("1. 並列処理版を使用（推奨）:")
    print("   - セル07-BATCH: 10件ずつバッチ処理")
    print("   - セル07-PARALLEL: 並列処理最適化版")
    print("")
    print("2. または、このセルを改造して全件処理:")
    print("   for idx, row in false_negatives_df.iterrows():")
    print("       # 処理...")
    
else:
    print("❌ テストが実行されていません")
    print("   上記のエラーを解決してください")

# ========== 8. GPU使用状況の確認 ==========

print("\n" + "=" * 80)
print("🖥️ GPU使用状況")
print("=" * 80)

print("\n📍 GPU使用場所:")
print("  - ローカル: XGBoostモデル（予測時）")
print("  - リモート: vLLMサーバー（192.168.100.71）")
print("    → Qwen3-14Bモデルの推論にGPUを使用")
print("")
print("⚠️ 注意:")
print("  - このノートブックではGPU使用は見えません")
print("  - vLLMサーバー側でGPUが使用されています")
print("  - サーバーログでGPU使用状況を確認してください")

# グローバル変数として保存（必要に応じて）
if 'results' in locals():
    globals()['test_results'] = results
    globals()['test_results_df'] = results_df
    print("\n💾 変数を保存しました:")
    print("  - test_results: 結果リスト")
    print("  - test_results_df: 結果DataFrame")


# In[ ]:





# In[24]:


"""
プログラム名: 03_ai_agent_analysis.ipynb
概要: MLモデルが見逃したフィッシングサイトをAIエージェント（Qwen3-14B）で検出
目的: 国際学会論文での発表（MLとLLMのハイブリッドアプローチの実証）
作成日: 2025-01-17
"""

"""
セル番号: 08-INVESTIGATE
概要: kobe-denki.co.jpの偽陽性問題を調査
入力: DANGEROUS_TLDS, HIGH_RISK_WORDS, false_negatives_df
出力: 問題の原因分析
"""

print("=" * 80)
print("🔍 偽陽性問題の調査: kobe-denki.co.jp")
print("=" * 80)

# ========== 1. DANGEROUS_TLDSの確認 ==========

print("\n1️⃣ DANGEROUS_TLDSの内容確認:")
print("-" * 40)

if 'DANGEROUS_TLDS' in globals():
    print(f"DANGEROUS_TLDS ({len(DANGEROUS_TLDS)}個):")
    # .jpや.co.jpが含まれているか確認
    jp_related = [tld for tld in DANGEROUS_TLDS if 'jp' in tld or 'co' in tld]
    if jp_related:
        print(f"\n⚠️ 日本関連TLDが危険リストに含まれています:")
        for tld in jp_related:
            print(f"   - {tld}")
    else:
        print("✅ .jpや.co.jpは危険TLDリストに含まれていません")
    
    # 実際のリスト内容（最初の20個）
    print(f"\n危険TLDリスト（最初の20個）:")
    for i, tld in enumerate(DANGEROUS_TLDS[:20], 1):
        print(f"   {i:2d}. {tld}")
else:
    print("❌ DANGEROUS_TLDSが未定義です")
    print("   セル04で統計的に生成されるはずです")

# ========== 2. HIGH_RISK_WORDSの確認 ==========

print("\n2️⃣ HIGH_RISK_WORDSの内容確認:")
print("-" * 40)

if 'HIGH_RISK_WORDS' in globals():
    print(f"HIGH_RISK_WORDS ({len(HIGH_RISK_WORDS)}個):")
    
    # 'co'や'jp'が単語として含まれているか確認
    problematic_words = [word for word in HIGH_RISK_WORDS if word in ['co', 'jp', 'com']]
    if problematic_words:
        print(f"\n❌ 問題のある単語が高リスクリストに含まれています:")
        for word in problematic_words:
            print(f"   - '{word}' ← これが原因の可能性！")
    else:
        print("✅ 'co', 'jp'などは高リスク単語に含まれていません")
    
    # 実際のリスト内容（最初の20個）
    print(f"\n高リスク単語リスト（最初の20個）:")
    for i, word in enumerate(HIGH_RISK_WORDS[:20], 1):
        print(f"   {i:2d}. {word}")
else:
    print("❌ HIGH_RISK_WORDSが未定義です")

# ========== 3. kobe-denki.co.jpの実際のステータス確認 ==========

print("\n3️⃣ kobe-denki.co.jpの実際のステータス:")
print("-" * 40)

if 'false_negatives_df' in globals():
    # このドメインが本当に偽陰性（フィッシング）なのか確認
    kobe_rows = false_negatives_df[false_negatives_df['domain'].str.contains('kobe-denki', na=False)]
    
    if not kobe_rows.empty:
        print("📊 データベース内の情報:")
        for idx, row in kobe_rows.iterrows():
            print(f"   - ドメイン: {row['domain']}")
            if 'prediction_proba' in row:
                print(f"   - ML予測確率: {row['prediction_proba']:.3f}")
            if 'is_phishing' in row:
                print(f"   - 実際のラベル: {'フィッシング' if row['is_phishing'] else '正常'}")
            print(f"   - データソース: 偽陰性リスト（MLが見逃したフィッシング）")
        
        print("\n⚠️ 重要な観察:")
        print("   このドメインは偽陰性リストに含まれているため、")
        print("   実際にはフィッシングサイトである可能性があります。")
        print("   ただし、.co.jpドメインは日本企業の正規ドメインなので、")
        print("   データラベルが間違っている可能性も考慮すべきです。")
    else:
        print("kobe-denki.co.jpはデータベースに見つかりません")
else:
    print("false_negatives_dfが未定義です")

# ========== 4. 問題の分析 ==========

print("\n4️⃣ 問題の分析:")
print("-" * 40)

print("\n🔍 発見された問題:")

problems = []

# 問題1: 'co'が高リスク単語として扱われている可能性
if 'HIGH_RISK_WORDS' in globals() and 'co' in HIGH_RISK_WORDS:
    problems.append("""
❌ 問題1: 'co'が高リスク単語として登録されている
   - 'co'は.co.jpや.co.ukなどの正規TLDの一部
   - これが原因で正規の企業ドメインが誤判定される
   - 解決策: HIGH_RISK_WORDSから'co'を除外する必要がある
""")

# 問題2: 単語抽出ロジックの問題
problems.append("""
⚠️ 可能性2: ドメイン解析ロジックの問題
   - 'kobe-denki.co.jp'が'kobe', 'denki', 'co', 'jp'に分割される
   - 'co'が独立した単語として評価されている
   - 解決策: TLD部分（.co.jp）を単語抽出から除外すべき
""")

# 問題3: contextual_risk_assessmentの誤判定
problems.append("""
⚠️ 可能性3: contextual_risk_assessmentツールの誤判定
   - 低ML確率（0.144）を「逆説的に危険」と解釈
   - .co.jpを危険と誤認識
   - 解決策: 日本の正規TLD（.co.jp, .ne.jp等）を安全リストに追加
""")

for problem in problems:
    print(problem)

# ========== 5. 修正提案 ==========

print("\n5️⃣ 修正提案:")
print("-" * 40)

print("""
📝 推奨される修正:

1. **HIGH_RISK_WORDSのフィルタリング**:
   ```python
   # 問題のある単語を除外
   problematic = ['co', 'com', 'jp', 'net', 'org']
   HIGH_RISK_WORDS = [w for w in HIGH_RISK_WORDS if w not in problematic]
   ```

2. **正規TLDの安全リスト作成**:
   ```python
   SAFE_TLDS = [
       '.co.jp',  # 日本企業
       '.ne.jp',  # 日本ネットワーク
       '.or.jp',  # 日本組織
       '.ac.jp',  # 日本教育機関
       '.go.jp',  # 日本政府
       '.edu',    # 教育機関
       '.gov',    # 政府機関
   ]
   ```

3. **ドメイン評価ロジックの改善**:
   - TLD部分を単語抽出から除外
   - 正規TLDの場合はペナルティを減らす
   - 証明書に組織情報がある場合は信頼度を上げる
""")

# ========== 6. テスト: 修正版の単語リスト ==========

print("\n6️⃣ 修正版の単語リストを作成:")
print("-" * 40)

if 'HIGH_RISK_WORDS' in globals():
    # 問題のある単語を除外
    problematic = ['co', 'com', 'jp', 'net', 'org', 'www']
    HIGH_RISK_WORDS_FIXED = [w for w in HIGH_RISK_WORDS if w not in problematic]
    
    removed = set(HIGH_RISK_WORDS) - set(HIGH_RISK_WORDS_FIXED)
    print(f"除外された単語: {removed}")
    print(f"修正前: {len(HIGH_RISK_WORDS)}個")
    print(f"修正後: {len(HIGH_RISK_WORDS_FIXED)}個")
    
    # グローバル変数として保存
    globals()['HIGH_RISK_WORDS_FIXED'] = HIGH_RISK_WORDS_FIXED
    
    print("\n✅ HIGH_RISK_WORDS_FIXEDを作成しました")
    print("   これを使用することで偽陽性を減らせます")

# ========== 7. 再評価テスト ==========

print("\n7️⃣ kobe-denki.co.jpの再評価（可能な場合）:")
print("-" * 40)

def analyze_domain_parts(domain):
    """ドメインの構成要素を分析"""
    parts = domain.split('.')
    
    print(f"ドメイン: {domain}")
    print(f"  - 全体の長さ: {len(domain)}文字")
    print(f"  - パーツ: {parts}")
    
    # TLD判定
    if len(parts) >= 2:
        if len(parts) >= 3 and parts[-2] in ['co', 'ac', 'or', 'ne', 'go']:
            tld = f".{parts[-2]}.{parts[-1]}"
            domain_without_tld = '.'.join(parts[:-2])
        else:
            tld = f".{parts[-1]}"
            domain_without_tld = '.'.join(parts[:-1])
        
        print(f"  - TLD: {tld}")
        print(f"  - ドメイン名部分: {domain_without_tld}")
        
        # 日本の正規TLDかチェック
        jp_official_tlds = ['.co.jp', '.ne.jp', '.or.jp', '.ac.jp', '.go.jp']
        if tld in jp_official_tlds:
            print(f"  ✅ {tld}は日本の正規企業/組織向けTLDです")
            print(f"     → このドメインは正当な可能性が高い")
        
        # 単語抽出（ハイフンで分割）
        words = domain_without_tld.replace('-', ' ').replace('_', ' ').split()
        print(f"  - 抽出された単語: {words}")
        
        # 'kobe'は神戸、'denki'は電機/電気
        if 'kobe' in words:
            print("  - 'kobe' = 神戸（日本の都市名）")
        if 'denki' in words:
            print("  - 'denki' = 電機/電気（日本語）")

# テスト実行
analyze_domain_parts('kobe-denki.co.jp')

print("\n" + "=" * 80)
print("📊 結論:")
print("=" * 80)
print("""
kobe-denki.co.jpが誤判定される原因:
1. 'co'が高リスク単語として登録されている（最も可能性が高い）
2. .co.jpが危険TLDとして誤認識されている
3. 日本の正規企業ドメインの特徴が考慮されていない

推奨対応:
- HIGH_RISK_WORDSから'co'を除外
- 日本の正規TLD（.co.jp等）を安全リストとして扱う
- 証明書に組織情報がある場合は信頼度を上げる
""")


# In[ ]:





# In[ ]:

🧪 シンプル動作テスト（5件のみ）

1️⃣ vLLMサーバー接続確認:
----------------------------------------
❌ VLLM_CONFIG が未定義
   セル06を実行してください

2️⃣ 評価関数の確認:
----------------------------------------
❌ evaluate_domain が未定義です
   セル06を実行してください

3️⃣ テストデータ準備:
----------------------------------------
❌ false_negatives_df が未定義です
   セル01または02でデータを読み込んでください

4️⃣ 評価実行（1件ずつ処理）:
----------------------------------------

❌ テストを実行できません
   必要な変数または関数が未定義です

💡 次のステップ:
----------------------------------------
❌ テストが実行されていません
   上記のエラーを解決してください

🖥️ GPU使用状況

📍 GPU使用場所:
  - ローカル: XGBoostモデル（予測時）
  - リモート: vLLMサーバー（192.168.100.71）
    → Qwen3-14Bモデルの推論にGPUを使用

⚠️ 注意:
  - このノートブックではGPU使用は見えません
  - vLLMサーバー側でGPUが使用されています
  - サーバーログでGPU使用状況を確認してください
🔍 偽陽性問題の調査: kobe-denki.co.jp

1️⃣ DANGEROUS_TLDSの内容確認:
----------------------------------------
❌ DANGEROUS_TLDSが未定義です
   セル04で統計的に生成されるはずです

2️⃣ HIGH_RISK_WORDSの内容確認:
----------------------------------------
❌ HIGH_RISK_WORDSが未定義です

3️⃣ kobe-denki.co.jpの実際のステータス:
----------------------------

In [7]:
"""
プログラム名: 03_ai_agent_analysis.ipynb
概要: MLモデルが見逃したフィッシングサイトをAIエージェント（Qwen3-14B）で検出
目的: 国際学会論文での発表（MLとLLMのハイブリッドアプローチの実証）
作成日: 2025-01-17
"""

"""
セル番号: 07-RERUN-FIX
概要: process_domains_batch_fixed関数の呼び出し方法を修正
入力: evaluate_domain, process_domains_batch_fixed
出力: 修正版のevaluate_with_fixed_agent関数
"""

import asyncio
import time
import pandas as pd
from typing import List, Dict, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed

print("=" * 70)
print("🔧 セル07-RERUN用の関数修正")
print("=" * 70)

# ========== 1. 関数のシグネチャを確認 ==========

print("\n📋 関数シグネチャの確認:")

if 'process_domains_batch_fixed' in globals():
    import inspect
    sig = inspect.signature(process_domains_batch_fixed)
    print(f"✅ process_domains_batch_fixed のシグネチャ:")
    print(f"   {sig}")
    
    # パラメータ詳細
    print("\n   パラメータ:")
    for param_name, param in sig.parameters.items():
        default = param.default
        if default == inspect.Parameter.empty:
            default = "必須"
        print(f"   - {param_name}: {default}")
else:
    print("❌ process_domains_batch_fixed が未定義です")
    print("   セル06を実行してください")

# ========== 2. 修正版のevaluate_with_fixed_agent関数 ==========

async def evaluate_with_fixed_agent_corrected(samples_df: pd.DataFrame, 
                                              max_concurrent: int = 10) -> pd.DataFrame:
    """
    修正版エージェントで評価（関数呼び出し修正版）
    
    Args:
        samples_df: 評価対象のDataFrame
        max_concurrent: 最大並列数
    
    Returns:
        評価結果のDataFrame
    """
    
    if samples_df.empty:
        print("❌ No samples to evaluate")
        return pd.DataFrame()
    
    # ML確率カラムの確認
    prob_col = 'prediction_proba' if 'prediction_proba' in samples_df.columns else 'ml_probability'
    
    # データの準備
    domains = samples_df['domain'].tolist()
    ml_probabilities = samples_df[prob_col].tolist()
    
    print(f"\n⚡ 評価開始:")
    print(f"  - 対象: {len(domains)}件")
    print(f"  - 並列数: {max_concurrent}")
    
    start_time = time.time()
    
    # process_domains_batch_fixedは同期関数なので、ThreadPoolExecutorで非同期化
    loop = asyncio.get_event_loop()
    
    # 修正版関数を使用（正しいパラメータで）
    if 'process_domains_batch_fixed' in globals():
        print("  - 使用: process_domains_batch_fixed（修正版・同期）")
        
        # ThreadPoolExecutorで非同期実行
        with ThreadPoolExecutor(max_workers=1) as executor:
            future = loop.run_in_executor(
                executor,
                process_domains_batch_fixed,
                domains,           # 第1引数: domains
                ml_probabilities,  # 第2引数: ml_probabilities
                max_concurrent,    # 第3引数: batch_size（max_concurrentを使用）
                False             # 第4引数: enable_trace
            )
            results = await future
            
    elif 'process_domains_batch' in globals():
        print("  - 使用: process_domains_batch（元版・非同期）")
        
        # (domain, ml_probability)のタプルリストを作成
        domains_data = list(zip(domains, ml_probabilities))
        
        # 非同期版を直接呼び出し
        results = await process_domains_batch(domains_data, max_concurrent=max_concurrent)
    else:
        print("❌ 評価関数が見つかりません")
        return pd.DataFrame()
    
    elapsed = time.time() - start_time
    
    # 結果をDataFrameに変換
    results_df = pd.DataFrame(results)
    
    # 元のDataFrameの情報を追加
    if not results_df.empty and not samples_df.empty:
        # インデックスを合わせる
        results_df.index = samples_df.index[:len(results_df)]
        
        # 元のカラムを追加（重複を避ける）
        for col in samples_df.columns:
            if col not in results_df.columns:
                results_df[col] = samples_df[col]
    
    print(f"\n✅ 評価完了:")
    print(f"  - 処理時間: {elapsed:.2f}秒")
    print(f"  - 処理速度: {len(domains)/elapsed:.2f} ドメイン/秒")
    
    return results_df

# ========== 3. 別の実装方法（より簡単） ==========

def evaluate_samples_sync(samples_df: pd.DataFrame, 
                         batch_size: int = 24) -> pd.DataFrame:
    """
    同期的にサンプルを評価（シンプル版）
    
    Args:
        samples_df: 評価対象のDataFrame
        batch_size: バッチサイズ
    
    Returns:
        評価結果のDataFrame
    """
    
    if samples_df.empty:
        print("❌ No samples to evaluate")
        return pd.DataFrame()
    
    # ML確率カラムの確認
    prob_col = 'prediction_proba' if 'prediction_proba' in samples_df.columns else 'ml_probability'
    
    # データの準備
    domains = samples_df['domain'].tolist()
    ml_probabilities = samples_df[prob_col].tolist()
    
    print(f"\n⚡ 同期評価開始:")
    print(f"  - 対象: {len(domains)}件")
    print(f"  - バッチサイズ: {batch_size}")
    
    start_time = time.time()
    
    # 評価実行
    if 'process_domains_batch_fixed' in globals():
        results = process_domains_batch_fixed(
            domains, 
            ml_probabilities, 
            batch_size=batch_size,
            enable_trace=False
        )
    elif 'evaluate_domain' in globals():
        # フォールバック: 個別に評価
        print("  ⚠️ process_domains_batch_fixedが未定義。個別評価にフォールバック")
        results = []
        for domain, ml_prob in zip(domains, ml_probabilities):
            result = evaluate_domain(domain, ml_prob, enable_trace=False)
            results.append(result)
    else:
        print("❌ 評価関数が見つかりません")
        return pd.DataFrame()
    
    elapsed = time.time() - start_time
    
    # 結果をDataFrameに変換
    results_df = pd.DataFrame(results)
    
    print(f"\n✅ 評価完了:")
    print(f"  - 処理時間: {elapsed:.2f}秒")
    print(f"  - 処理速度: {len(domains)/elapsed:.2f} ドメイン/秒")
    
    return results_df

# ========== 4. グローバル変数として保存 ==========

globals()['evaluate_with_fixed_agent_corrected'] = evaluate_with_fixed_agent_corrected
globals()['evaluate_samples_sync'] = evaluate_samples_sync

# 元の関数を上書き（セル07-RERUNで使用される）
globals()['evaluate_with_fixed_agent'] = evaluate_with_fixed_agent_corrected

print("\n" + "=" * 70)
print("✅ 関数修正完了")
print("=" * 70)

print("\n📋 修正内容:")
print("  1. process_domains_batch_fixed の正しい呼び出し方法")
print("     - awaitを使わない（同期関数のため）")
print("     - max_concurrent → batch_size パラメータ")
print("     - ThreadPoolExecutorで非同期化")

print("\n📋 提供する関数:")
print("  - evaluate_with_fixed_agent_corrected: 非同期版（修正済み）")
print("  - evaluate_samples_sync: 同期版（シンプル）")

print("\n🎯 推奨される使用方法:")
print("  1. 非同期環境（Jupyter）:")
print("     results_df = await evaluate_with_fixed_agent_corrected(samples_df)")
print("  2. 同期環境:")
print("     results_df = evaluate_samples_sync(samples_df)")

# ========== 5. テスト ==========

def test_evaluation_functions():
    """評価関数のテスト"""
    print("\n" + "=" * 70)
    print("🧪 評価関数のテスト")
    print("=" * 70)
    
    # テストデータ作成
    test_data = pd.DataFrame({
        'domain': ['test-phishing.com', 'normal-site.org'],
        'prediction_proba': [0.15, 0.85]
    })
    
    print("\n📝 テストデータ:")
    print(test_data)
    
    try:
        # 同期版テスト
        print("\n1️⃣ 同期版テスト:")
        results = evaluate_samples_sync(test_data, batch_size=2)
        if not results.empty:
            print("✅ 成功")
            print(f"   結果: {len(results)}件")
        else:
            print("❌ 失敗: 空の結果")
            
    except Exception as e:
        print(f"❌ エラー: {e}")
    
    print("\n✅ テスト完了")

# テスト実行の提案
print("\n💡 テストを実行するには:")
print("   test_evaluation_functions()")
print("\nまたは、セル07-RERUNを再実行してください")

🔧 セル07-RERUN用の関数修正

📋 関数シグネチャの確認:
❌ process_domains_batch_fixed が未定義です
   セル06を実行してください

✅ 関数修正完了

📋 修正内容:
  1. process_domains_batch_fixed の正しい呼び出し方法
     - awaitを使わない（同期関数のため）
     - max_concurrent → batch_size パラメータ
     - ThreadPoolExecutorで非同期化

📋 提供する関数:
  - evaluate_with_fixed_agent_corrected: 非同期版（修正済み）
  - evaluate_samples_sync: 同期版（シンプル）

🎯 推奨される使用方法:
  1. 非同期環境（Jupyter）:
     results_df = await evaluate_with_fixed_agent_corrected(samples_df)
  2. 同期環境:
     results_df = evaluate_samples_sync(samples_df)

💡 テストを実行するには:
   test_evaluation_functions()

または、セル07-RERUNを再実行してください


In [8]:
"""
プログラム名: 03_ai_agent_analysis.ipynb
概要: MLモデルが見逃したフィッシングサイトをAIエージェント（Qwen3-14B）で検出
目的: 国際学会論文での発表（MLとLLMのハイブリッドアプローチの実証）
作成日: 2025-01-17
"""

"""
セル番号: 07-SIMPLE
概要: シンプルな動作確認版（まず5件でテスト）
入力: false_negatives_df, evaluate_domain
出力: テスト結果の表示と動作確認
"""

import time
import pandas as pd
from datetime import datetime

print("=" * 80)
print("🧪 シンプル動作テスト（5件のみ）")
print("=" * 80)

# ========== 1. vLLMサーバー接続確認 ==========

print("\n1️⃣ vLLMサーバー接続確認:")
print("-" * 40)

try:
    from langchain_openai import ChatOpenAI
    
    # vLLM設定の確認
    if 'VLLM_CONFIG' in globals():
        print(f"✅ VLLM設定:")
        print(f"   - URL: {VLLM_CONFIG['base_url']}")
        print(f"   - Model: {VLLM_CONFIG['model']}")
        
        # 簡単なテスト
        test_llm = ChatOpenAI(**VLLM_CONFIG)
        response = test_llm.invoke("Say 'OK' if you are working")
        print(f"✅ vLLMサーバー応答確認")
        print(f"   サーバー側でGPUが使用されています")
    else:
        print("❌ VLLM_CONFIG が未定義")
        print("   セル06を実行してください")
        
except Exception as e:
    print(f"❌ vLLMサーバー接続エラー: {e}")

# ========== 2. 評価関数の確認 ==========

print("\n2️⃣ 評価関数の確認:")
print("-" * 40)

if 'evaluate_domain' in globals():
    print("✅ evaluate_domain 関数が定義されています")
else:
    print("❌ evaluate_domain が未定義です")
    print("   セル06を実行してください")

# ========== 3. テストデータの準備 ==========

print("\n3️⃣ テストデータ準備:")
print("-" * 40)

if 'false_negatives_df' not in globals():
    print("❌ false_negatives_df が未定義です")
    print("   セル01または02でデータを読み込んでください")
else:
    # 5件だけ取得
    test_samples = false_negatives_df.head(5).copy()
    print(f"✅ テストサンプル: {len(test_samples)}件")
    
    # ML確率カラムの確認
    prob_col = 'prediction_proba' if 'prediction_proba' in test_samples.columns else 'ml_probability'
    
    print("\n📋 テストドメイン:")
    for idx, row in test_samples.iterrows():
        print(f"   {idx+1}. {row['domain'][:50]:50s} (ML: {row[prob_col]:.3f})")

# ========== 4. 1件ずつ評価（最もシンプル） ==========

print("\n4️⃣ 評価実行（1件ずつ処理）:")
print("-" * 40)

def simple_evaluate_one(domain, ml_prob):
    """1件だけ評価する最もシンプルな関数"""
    
    print(f"\n評価中: {domain}")
    print(f"  ML確率: {ml_prob:.3f}")
    
    start = time.time()
    
    try:
        # evaluate_domain を呼び出し
        result = evaluate_domain(domain, ml_prob, enable_trace=False)
        
        elapsed = time.time() - start
        
        # 結果の表示
        is_phishing = result.get('ai_is_phishing', False)
        confidence = result.get('ai_confidence', 0.0)
        
        print(f"  結果: {'🚨 フィッシング' if is_phishing else '✅ 正常'}")
        print(f"  信頼度: {confidence:.2f}")
        print(f"  処理時間: {elapsed:.1f}秒")
        
        # 詳細情報
        if result.get('detected_brands'):
            print(f"  検出ブランド: {', '.join(result['detected_brands'])}")
        if result.get('risk_factors'):
            print(f"  リスク要因: {', '.join(result['risk_factors'][:3])}")
        
        return result
        
    except Exception as e:
        print(f"  ❌ エラー: {e}")
        return {
            'domain': domain,
            'ml_probability': ml_prob,
            'ai_is_phishing': False,
            'ai_confidence': 0.0,
            'error': str(e)
        }

# ========== 5. テスト実行 ==========

if 'test_samples' in locals() and 'evaluate_domain' in globals():
    print("\n🚀 テスト評価を開始します...")
    print("=" * 80)
    
    results = []
    total_start = time.time()
    
    # 1件ずつ処理
    for idx, row in test_samples.iterrows():
        domain = row['domain']
        ml_prob = row[prob_col]
        
        result = simple_evaluate_one(domain, ml_prob)
        results.append(result)
        
        # 少し待機（サーバー負荷軽減）
        time.sleep(0.5)
    
    total_elapsed = time.time() - total_start
    
    # ========== 6. 結果サマリー ==========
    
    print("\n" + "=" * 80)
    print("📊 テスト結果サマリー")
    print("=" * 80)
    
    # 統計
    detected = sum(1 for r in results if r.get('ai_is_phishing', False))
    errors = sum(1 for r in results if r.get('error'))
    
    print(f"\n統計:")
    print(f"  - 処理件数: {len(results)}件")
    print(f"  - フィッシング検出: {detected}件 ({detected/len(results)*100:.1f}%)")
    print(f"  - エラー: {errors}件")
    print(f"  - 総処理時間: {total_elapsed:.1f}秒")
    print(f"  - 平均処理時間: {total_elapsed/len(results):.1f}秒/件")
    
    # 結果詳細
    print(f"\n詳細結果:")
    for i, result in enumerate(results, 1):
        domain = test_samples.iloc[i-1]['domain']
        is_phishing = "🚨 フィッシング" if result.get('ai_is_phishing') else "✅ 正常"
        confidence = result.get('ai_confidence', 0.0)
        print(f"  {i}. {domain[:40]:40s} → {is_phishing} (信頼度: {confidence:.2f})")
    
    # DataFrameに変換
    results_df = pd.DataFrame(results)
    
    print("\n" + "=" * 80)
    print("✅ テスト完了！")
    print("=" * 80)
    
else:
    print("\n❌ テストを実行できません")
    print("   必要な変数または関数が未定義です")

# ========== 7. 次のステップ ==========

print("\n💡 次のステップ:")
print("-" * 40)

if 'results' in locals() and len(results) > 0:
    avg_time = total_elapsed / len(results)
    estimated_total = 4215 * avg_time / 60
    
    print(f"✅ テスト成功！")
    print(f"   平均処理時間: {avg_time:.1f}秒/件")
    print(f"   4,215件の推定処理時間: {estimated_total:.1f}分")
    
    print("\n全件処理を開始する場合:")
    print("1. 並列処理版を使用（推奨）:")
    print("   - セル07-BATCH: 10件ずつバッチ処理")
    print("   - セル07-PARALLEL: 並列処理最適化版")
    print("")
    print("2. または、このセルを改造して全件処理:")
    print("   for idx, row in false_negatives_df.iterrows():")
    print("       # 処理...")
    
else:
    print("❌ テストが実行されていません")
    print("   上記のエラーを解決してください")

# ========== 8. GPU使用状況の確認 ==========

print("\n" + "=" * 80)
print("🖥️ GPU使用状況")
print("=" * 80)

print("\n📍 GPU使用場所:")
print("  - ローカル: XGBoostモデル（予測時）")
print("  - リモート: vLLMサーバー（192.168.100.71）")
print("    → Qwen3-14Bモデルの推論にGPUを使用")
print("")
print("⚠️ 注意:")
print("  - このノートブックではGPU使用は見えません")
print("  - vLLMサーバー側でGPUが使用されています")
print("  - サーバーログでGPU使用状況を確認してください")

# グローバル変数として保存（必要に応じて）
if 'results' in locals():
    globals()['test_results'] = results
    globals()['test_results_df'] = results_df
    print("\n💾 変数を保存しました:")
    print("  - test_results: 結果リスト")
    print("  - test_results_df: 結果DataFrame")


# In[ ]:





# In[24]:


"""
プログラム名: 03_ai_agent_analysis.ipynb
概要: MLモデルが見逃したフィッシングサイトをAIエージェント（Qwen3-14B）で検出
目的: 国際学会論文での発表（MLとLLMのハイブリッドアプローチの実証）
作成日: 2025-01-17
"""

"""
セル番号: 08-INVESTIGATE
概要: kobe-denki.co.jpの偽陽性問題を調査
入力: DANGEROUS_TLDS, HIGH_RISK_WORDS, false_negatives_df
出力: 問題の原因分析
"""

print("=" * 80)
print("🔍 偽陽性問題の調査: kobe-denki.co.jp")
print("=" * 80)

# ========== 1. DANGEROUS_TLDSの確認 ==========

print("\n1️⃣ DANGEROUS_TLDSの内容確認:")
print("-" * 40)

if 'DANGEROUS_TLDS' in globals():
    print(f"DANGEROUS_TLDS ({len(DANGEROUS_TLDS)}個):")
    # .jpや.co.jpが含まれているか確認
    jp_related = [tld for tld in DANGEROUS_TLDS if 'jp' in tld or 'co' in tld]
    if jp_related:
        print(f"\n⚠️ 日本関連TLDが危険リストに含まれています:")
        for tld in jp_related:
            print(f"   - {tld}")
    else:
        print("✅ .jpや.co.jpは危険TLDリストに含まれていません")
    
    # 実際のリスト内容（最初の20個）
    print(f"\n危険TLDリスト（最初の20個）:")
    for i, tld in enumerate(DANGEROUS_TLDS[:20], 1):
        print(f"   {i:2d}. {tld}")
else:
    print("❌ DANGEROUS_TLDSが未定義です")
    print("   セル04で統計的に生成されるはずです")

# ========== 2. HIGH_RISK_WORDSの確認 ==========

print("\n2️⃣ HIGH_RISK_WORDSの内容確認:")
print("-" * 40)

if 'HIGH_RISK_WORDS' in globals():
    print(f"HIGH_RISK_WORDS ({len(HIGH_RISK_WORDS)}個):")
    
    # 'co'や'jp'が単語として含まれているか確認
    problematic_words = [word for word in HIGH_RISK_WORDS if word in ['co', 'jp', 'com']]
    if problematic_words:
        print(f"\n❌ 問題のある単語が高リスクリストに含まれています:")
        for word in problematic_words:
            print(f"   - '{word}' ← これが原因の可能性！")
    else:
        print("✅ 'co', 'jp'などは高リスク単語に含まれていません")
    
    # 実際のリスト内容（最初の20個）
    print(f"\n高リスク単語リスト（最初の20個）:")
    for i, word in enumerate(HIGH_RISK_WORDS[:20], 1):
        print(f"   {i:2d}. {word}")
else:
    print("❌ HIGH_RISK_WORDSが未定義です")

# ========== 3. kobe-denki.co.jpの実際のステータス確認 ==========

print("\n3️⃣ kobe-denki.co.jpの実際のステータス:")
print("-" * 40)

if 'false_negatives_df' in globals():
    # このドメインが本当に偽陰性（フィッシング）なのか確認
    kobe_rows = false_negatives_df[false_negatives_df['domain'].str.contains('kobe-denki', na=False)]
    
    if not kobe_rows.empty:
        print("📊 データベース内の情報:")
        for idx, row in kobe_rows.iterrows():
            print(f"   - ドメイン: {row['domain']}")
            if 'prediction_proba' in row:
                print(f"   - ML予測確率: {row['prediction_proba']:.3f}")
            if 'is_phishing' in row:
                print(f"   - 実際のラベル: {'フィッシング' if row['is_phishing'] else '正常'}")
            print(f"   - データソース: 偽陰性リスト（MLが見逃したフィッシング）")
        
        print("\n⚠️ 重要な観察:")
        print("   このドメインは偽陰性リストに含まれているため、")
        print("   実際にはフィッシングサイトである可能性があります。")
        print("   ただし、.co.jpドメインは日本企業の正規ドメインなので、")
        print("   データラベルが間違っている可能性も考慮すべきです。")
    else:
        print("kobe-denki.co.jpはデータベースに見つかりません")
else:
    print("false_negatives_dfが未定義です")

# ========== 4. 問題の分析 ==========

print("\n4️⃣ 問題の分析:")
print("-" * 40)

print("\n🔍 発見された問題:")

problems = []

# 問題1: 'co'が高リスク単語として扱われている可能性
if 'HIGH_RISK_WORDS' in globals() and 'co' in HIGH_RISK_WORDS:
    problems.append("""
❌ 問題1: 'co'が高リスク単語として登録されている
   - 'co'は.co.jpや.co.ukなどの正規TLDの一部
   - これが原因で正規の企業ドメインが誤判定される
   - 解決策: HIGH_RISK_WORDSから'co'を除外する必要がある
""")

# 問題2: 単語抽出ロジックの問題
problems.append("""
⚠️ 可能性2: ドメイン解析ロジックの問題
   - 'kobe-denki.co.jp'が'kobe', 'denki', 'co', 'jp'に分割される
   - 'co'が独立した単語として評価されている
   - 解決策: TLD部分（.co.jp）を単語抽出から除外すべき
""")

# 問題3: contextual_risk_assessmentの誤判定
problems.append("""
⚠️ 可能性3: contextual_risk_assessmentツールの誤判定
   - 低ML確率（0.144）を「逆説的に危険」と解釈
   - .co.jpを危険と誤認識
   - 解決策: 日本の正規TLD（.co.jp, .ne.jp等）を安全リストに追加
""")

for problem in problems:
    print(problem)

# ========== 5. 修正提案 ==========

print("\n5️⃣ 修正提案:")
print("-" * 40)

print("""
📝 推奨される修正:

1. **HIGH_RISK_WORDSのフィルタリング**:
   ```python
   # 問題のある単語を除外
   problematic = ['co', 'com', 'jp', 'net', 'org']
   HIGH_RISK_WORDS = [w for w in HIGH_RISK_WORDS if w not in problematic]
   ```

2. **正規TLDの安全リスト作成**:
   ```python
   SAFE_TLDS = [
       '.co.jp',  # 日本企業
       '.ne.jp',  # 日本ネットワーク
       '.or.jp',  # 日本組織
       '.ac.jp',  # 日本教育機関
       '.go.jp',  # 日本政府
       '.edu',    # 教育機関
       '.gov',    # 政府機関
   ]
   ```

3. **ドメイン評価ロジックの改善**:
   - TLD部分を単語抽出から除外
   - 正規TLDの場合はペナルティを減らす
   - 証明書に組織情報がある場合は信頼度を上げる
""")

# ========== 6. テスト: 修正版の単語リスト ==========

print("\n6️⃣ 修正版の単語リストを作成:")
print("-" * 40)

if 'HIGH_RISK_WORDS' in globals():
    # 問題のある単語を除外
    problematic = ['co', 'com', 'jp', 'net', 'org', 'www']
    HIGH_RISK_WORDS_FIXED = [w for w in HIGH_RISK_WORDS if w not in problematic]
    
    removed = set(HIGH_RISK_WORDS) - set(HIGH_RISK_WORDS_FIXED)
    print(f"除外された単語: {removed}")
    print(f"修正前: {len(HIGH_RISK_WORDS)}個")
    print(f"修正後: {len(HIGH_RISK_WORDS_FIXED)}個")
    
    # グローバル変数として保存
    globals()['HIGH_RISK_WORDS_FIXED'] = HIGH_RISK_WORDS_FIXED
    
    print("\n✅ HIGH_RISK_WORDS_FIXEDを作成しました")
    print("   これを使用することで偽陽性を減らせます")

# ========== 7. 再評価テスト ==========

print("\n7️⃣ kobe-denki.co.jpの再評価（可能な場合）:")
print("-" * 40)

def analyze_domain_parts(domain):
    """ドメインの構成要素を分析"""
    parts = domain.split('.')
    
    print(f"ドメイン: {domain}")
    print(f"  - 全体の長さ: {len(domain)}文字")
    print(f"  - パーツ: {parts}")
    
    # TLD判定
    if len(parts) >= 2:
        if len(parts) >= 3 and parts[-2] in ['co', 'ac', 'or', 'ne', 'go']:
            tld = f".{parts[-2]}.{parts[-1]}"
            domain_without_tld = '.'.join(parts[:-2])
        else:
            tld = f".{parts[-1]}"
            domain_without_tld = '.'.join(parts[:-1])
        
        print(f"  - TLD: {tld}")
        print(f"  - ドメイン名部分: {domain_without_tld}")
        
        # 日本の正規TLDかチェック
        jp_official_tlds = ['.co.jp', '.ne.jp', '.or.jp', '.ac.jp', '.go.jp']
        if tld in jp_official_tlds:
            print(f"  ✅ {tld}は日本の正規企業/組織向けTLDです")
            print(f"     → このドメインは正当な可能性が高い")
        
        # 単語抽出（ハイフンで分割）
        words = domain_without_tld.replace('-', ' ').replace('_', ' ').split()
        print(f"  - 抽出された単語: {words}")
        
        # 'kobe'は神戸、'denki'は電機/電気
        if 'kobe' in words:
            print("  - 'kobe' = 神戸（日本の都市名）")
        if 'denki' in words:
            print("  - 'denki' = 電機/電気（日本語）")

# テスト実行
analyze_domain_parts('kobe-denki.co.jp')

print("\n" + "=" * 80)
print("📊 結論:")
print("=" * 80)
print("""
kobe-denki.co.jpが誤判定される原因:
1. 'co'が高リスク単語として登録されている（最も可能性が高い）
2. .co.jpが危険TLDとして誤認識されている
3. 日本の正規企業ドメインの特徴が考慮されていない

推奨対応:
- HIGH_RISK_WORDSから'co'を除外
- 日本の正規TLD（.co.jp等）を安全リストとして扱う
- 証明書に組織情報がある場合は信頼度を上げる
""")


# In[ ]:





# In[ ]:

🧪 シンプル動作テスト（5件のみ）

1️⃣ vLLMサーバー接続確認:
----------------------------------------
❌ VLLM_CONFIG が未定義
   セル06を実行してください

2️⃣ 評価関数の確認:
----------------------------------------
❌ evaluate_domain が未定義です
   セル06を実行してください

3️⃣ テストデータ準備:
----------------------------------------
❌ false_negatives_df が未定義です
   セル01または02でデータを読み込んでください

4️⃣ 評価実行（1件ずつ処理）:
----------------------------------------

❌ テストを実行できません
   必要な変数または関数が未定義です

💡 次のステップ:
----------------------------------------
❌ テストが実行されていません
   上記のエラーを解決してください

🖥️ GPU使用状況

📍 GPU使用場所:
  - ローカル: XGBoostモデル（予測時）
  - リモート: vLLMサーバー（192.168.100.71）
    → Qwen3-14Bモデルの推論にGPUを使用

⚠️ 注意:
  - このノートブックではGPU使用は見えません
  - vLLMサーバー側でGPUが使用されています
  - サーバーログでGPU使用状況を確認してください
🔍 偽陽性問題の調査: kobe-denki.co.jp

1️⃣ DANGEROUS_TLDSの内容確認:
----------------------------------------
❌ DANGEROUS_TLDSが未定義です
   セル04で統計的に生成されるはずです

2️⃣ HIGH_RISK_WORDSの内容確認:
----------------------------------------
❌ HIGH_RISK_WORDSが未定義です

3️⃣ kobe-denki.co.jpの実際のステータス:
----------------------------

In [9]:
# === Guard: ensure `load_configuration` exists (self-contained, _compat-first) ===
try:
    load_configuration  # noqa: F821
except NameError:
    import sys, json
    from pathlib import Path
    from typing import Optional, Dict, Any
    # ensure _compat on path
    if "_compat" not in sys.path:
        sys.path.append("_compat")
    # try to import from _compat/paths.py
    try:
        from paths import ensure_roots, load_config as _load_config  # resolved from _compat/paths.py
    except Exception as _e:
        raise ImportError("Failed to import _compat/paths.py as `paths`. Add `_compat` to sys.path or fix environment.") from _e
    try:
        ensure_roots()
    except Exception:
        pass

    def _deep_update(base: Dict[str, Any], upd: Optional[Dict[str, Any]]) -> Dict[str, Any]:
        if not isinstance(base, dict):
            return upd or {}
        if not isinstance(upd, dict):
            return base
        for k, v in upd.items():
            if isinstance(v, dict) and isinstance(base.get(k), dict):
                base[k] = _deep_update(base.get(k, {}), v)
            else:
                base[k] = v
        return base

    def load_configuration(cfg_override: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        # prefer _load_config(); fallback to _compat/config.json
        base = {}
        try:
            base = _load_config()
        except Exception:
            cfg_path = Path("_compat/config.json")
            if not cfg_path.exists():
                raise FileNotFoundError(f"Missing config: {cfg_path}")
            base = json.loads(cfg_path.read_text(encoding="utf-8"))
        return _deep_update(dict(base), cfg_override or {})
print("[GUARD] load_configuration is ready.")

[GUARD] load_configuration is ready.


In [10]:
# === Cell 25: Controller API 関数 (NEW) ===
from typing import Tuple, Dict, Any
import os, json, time, traceback
from pathlib import Path
import joblib
import pandas as pd

def _safe_get(d: dict, k: str):
    return d[k] if isinstance(d, dict) and k in d else None

def generalization_study(session_id: str, cfg_in: Dict[str, Any]) -> Tuple[str, Dict[str, str]]:
    """ReturnCode: "OK" | "NOT_FOUND" | "INVALID_INPUT" | "ERROR"""
    try:
        cfg_local = load_configuration(cfg_override=cfg_in or {})
        dev_mode = bool(cfg_local["system"]["development_mode"])
        sample_size = int(cfg_local["analysis"]["sample_size"]) if dev_mode else None

        handoff_path = None
        candidates = [
            os.path.join(globals().get("HANDOFF_DIR","handoff"), "03_ai_agent_analysis_part3.pkl"),
            "handoff/03_ai_agent_analysis_part3.pkl"
        ]
        for p in candidates:
            if os.path.exists(p):
                handoff_path = p; break
        if not handoff_path:
            return "NOT_FOUND", {"error": "handoff(part3) not found"}

        handoff = joblib.load(handoff_path)

        fn_df  = _safe_get(handoff, "false_negatives_df")
        brands = _safe_get(handoff, "brand_keywords")
        tlds_d = _safe_get(handoff, "DANGEROUS_TLDS")
        certs  = _safe_get(handoff, "cert_full_info_map")

        missing = []
        if fn_df is None or not hasattr(fn_df, "shape"): missing.append("false_negatives_df")
        if not brands or len(brands) < 64: missing.append("brand_keywords(>=64)")
        if not tlds_d: missing.append("DANGEROUS_TLDS")
        if certs is None: missing.append("cert_full_info_map")
        if missing:
            return "NOT_FOUND", {"error": f"handoff keys missing: {missing}"}

        target_df = fn_df.copy()
        if dev_mode:
            n = min(sample_size or 50, len(target_df))
            if "prediction_proba" in target_df.columns:
                low = target_df[target_df["prediction_proba"] < 0.2]
                rest = target_df[target_df["prediction_proba"] >= 0.2]
                take_low = min(int(n*0.6), len(low))
                take_rest = n - take_low
                frames = []
                if take_low>0 and len(low)>0:
                    frames.append(low.sample(n=take_low, random_state=42))
                if take_rest>0 and len(rest)>0:
                    frames.append(rest.sample(n=take_rest, random_state=42))
                target_df = pd.concat(frames, ignore_index=True) if frames else target_df.sample(n=n, random_state=42)
            else:
                target_df = target_df.sample(n=n, random_state=42)

        if "domain" not in target_df.columns:
            return "INVALID_INPUT", {"error": "false_negatives_df has no 'domain' column"}
        domains = target_df["domain"].astype(str).tolist()
        probs = (target_df["prediction_proba"].astype(float).tolist()
                 if "prediction_proba" in target_df.columns else [0.5]*len(domains))

        start = time.time()
        try:
            if "process_domains_batch" in globals():
                import asyncio
                async def _run():
                    return await process_domains_batch(list(zip(domains, probs)),
                                                       max_concurrent=cfg_local["llm"]["max_concurrent"])
                loop = asyncio.new_event_loop()
                asyncio.set_event_loop(loop)
                results = loop.run_until_complete(_run())
                loop.close()
            else:
                results = []
                for d, p in zip(domains, probs):
                    results.append(evaluate_domain(d, p))
        except Exception as e:
            print(f"⚠️ LLM error: {e} -> deterministic fallback")
            results = []
            for d, p in zip(domains, probs):
                is_phish = (p < 0.2)
                results.append({
                    'domain': d, 'ml_probability': p, 'ai_is_phishing': is_phish,
                    'ai_confidence': 0.6 if is_phish else 0.3, 'success': True,
                    'ai_risk_level': 'high' if is_phish else 'low', 'deterministic': True
                })
        elapsed = time.time() - start

        detected = sum(1 for r in results if isinstance(r, dict) and r.get("ai_is_phishing"))
        pr_summary = {
            "evaluated": len(results),
            "ai_detected": detected,
            "elapsed_sec": elapsed,
        }
        threshold_rec = {"recommended_threshold": 0.5, "rationale": "placeholder (optimize in report)"}

        run_tag = time.strftime("%Y%m%d_%H%M%S")
        base_results = Path(cfg_local["paths"]["results_dir"]) / str(session_id) / "generalization"
        base_logs    = Path(cfg_local["paths"]["logs_dir"]) / "generalization"
        base_results.mkdir(parents=True, exist_ok=True)
        base_logs.mkdir(parents=True, exist_ok=True)

        paths = {
            "generalization_results": str(base_results / f"generalization_results_{run_tag}.pkl"),
            "pr_analysis":            str(base_results / f"pr_analysis_{run_tag}.json"),
            "threshold_recommendation": str(base_results / f"threshold_recommendation_{run_tag}.json"),
            "comparison_report":      str(base_results / f"comparison_report_{run_tag}.html"),
            "evaluation_csv":         str(base_results / f"evaluation_results_{run_tag}.csv"),
            "logs":                   str(base_logs    / f"generalization_study_{run_tag}.log"),
        }

        import joblib as _joblib
        _joblib.dump({"results": results, "config": cfg_local}, paths["generalization_results"])
        Path(paths["pr_analysis"]).write_text(json.dumps(pr_summary, indent=2, ensure_ascii=False), encoding="utf-8")
        Path(paths["threshold_recommendation"]).write_text(json.dumps(threshold_rec, indent=2, ensure_ascii=False), encoding="utf-8")

        html = f"""
        <html><body>
        <h1>Generalization Study</h1>
        <p>session: {session_id}</p>
        <ul>
          <li>evaluated: {pr_summary['evaluated']}</li>
          <li>ai_detected: {pr_summary['ai_detected']}</li>
          <li>elapsed_sec: {pr_summary['elapsed_sec']:.2f}</li>
          <li>threshold recommendation: {threshold_rec['recommended_threshold']}</li>
        </ul>
        </body></html>
        """
        Path(paths["comparison_report"]).write_text(html, encoding="utf-8")

        try:
            pd.DataFrame(results).to_csv(paths["evaluation_csv"], index=False)
        except Exception:
            pass

        return "OK", paths

    except FileNotFoundError as e:
        return "NOT_FOUND", {"error": str(e)}
    except ValueError as e:
        return "INVALID_INPUT", {"error": str(e)}
    except Exception as e:
        import traceback as _tb
        return "ERROR", {"error": f"{e}", "trace": _tb.format_exc()[:2000]}


In [11]:

# Cell RUN: 評価実行ランナー
# - configオーバーライド（SAMPLES, MAX_WORKERS, SAMPLING）
# - generalization_study(session_id, cfg_in) を呼び出し
# - 実行後に結果ファイル/ハンドオフを保存（可能な限り）

from datetime import datetime
import os, json, pickle
from pathlib import Path
import pandas as pd

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# configの軽量オーバーライド辞書を構成
override = {
    "system": {
        "development_mode": bool(cfg["system"]["development_mode"]),
        "seed": int(cfg["system"].get("seed", 42))
    },
    "evaluation": {
        "samples": int(SAMPLES),
        "max_workers": int(MAX_WORKERS),
        "sampling": SAMPLING
    }
}

# 実行（03系のシグネチャ: generalization_study(session_id, cfg_in)）
try:
    ret = generalization_study(session_id=RUN_ID, cfg_in=override)
except NameError:
    print("[ERROR] generalization_study 関数が見つかりません。Cell 25の移植をご確認ください。")
    raise
except Exception as e:
    print("[ERROR] 評価実行で例外が発生:", e)
    raise

# 返り値の解釈: (ReturnCode, Paths) または dict
evaluation_results = None
performance_metrics = {}
error_analysis = {}
test_results = {}

if isinstance(ret, tuple) and len(ret) == 2:
    return_code, paths_map = ret
    print("ReturnCode:", return_code)
    print("Paths:", paths_map)
    # 既知の結果ファイルがパスに含まれていれば読込を試みる
    def _load_json(p):
        try:
            with open(p, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception:
            return None
    def _load_csv_df(p):
        try:
            return pd.read_csv(p)
        except Exception:
            return None
    # 期待されるキーの試行的読込
    if isinstance(paths_map, dict):
        evaluation_results = _load_csv_df(paths_map.get("evaluation_results_csv", "")) or evaluation_results
        performance_metrics = _load_json(paths_map.get("performance_metrics_json", "")) or performance_metrics
        error_analysis = _load_json(paths_map.get("error_analysis_json", "")) or error_analysis
        # その他の中間成果もあればここで追加読込可能
elif isinstance(ret, dict):
    evaluation_results = ret.get("evaluation_results")
    performance_metrics = ret.get("performance_metrics", {})
    error_analysis = ret.get("error_analysis", {})
    test_results = ret.get("test_results", {})
else:
    print("[WARN] 返り値の形式が未対応です:", type(ret))

# 結果出力パス
results_dir = Path(BASE_DIRS["results"])
handoff_dir = Path(BASE_DIRS["handoff"])

csv_path = results_dir / f"evaluation_results_{timestamp}.csv"
perf_path = results_dir / f"performance_metrics_{timestamp}.json"
err_path = results_dir / f"error_analysis_{timestamp}.json"
handoff_out = handoff_dir / "04-5_evaluation_results.pkl"

# 保存
try:
    if evaluation_results is not None and hasattr(evaluation_results, "to_csv"):
        evaluation_results.to_csv(csv_path, index=False)
        print("[OK] CSV saved:", csv_path)
    with open(perf_path, "w", encoding="utf-8") as f:
        json.dump(performance_metrics, f, ensure_ascii=False, indent=2)
        print("[OK] metrics saved:", perf_path)
    with open(err_path, "w", encoding="utf-8") as f:
        json.dump(error_analysis, f, ensure_ascii=False, indent=2)
        print("[OK] error_analysis saved:", err_path)
    # handoff pkl
    with open(handoff_out, "wb") as f:
        pickle.dump({
            "evaluation_results": evaluation_results,
            "performance_metrics": performance_metrics,
            "error_analysis": error_analysis,
            "test_results": test_results,
            "cfg": cfg,
            "override": override,
            "timestamp": timestamp,
            "run_id": RUN_ID
        }, f)
        print("[OK] handoff saved:", handoff_out)
except Exception as e:
    print("[ERROR] 結果保存エラー:", e)
    raise

print("DONE.")


ReturnCode: NOT_FOUND
Paths: {'error': 'handoff(part3) not found'}
[OK] metrics saved: artifacts/2025-10-22_074213/results/performance_metrics_20251026_180030.json
[OK] error_analysis saved: artifacts/2025-10-22_074213/results/error_analysis_20251026_180030.json
[OK] handoff saved: artifacts/2025-10-22_074213/handoff/04-5_evaluation_results.pkl
DONE.


In [12]:
# === C-04 Evaluation Compatibility Layer (04-5) ===
# Purpose: make 04-5 robust even if original cells are incomplete.
# This cell:
#  1) Resolves RUN_ID (latest artifacts/* that has 04-4 handoff)
#  2) Loads 04-4 handoff and 04-3 setup
#  3) Dynamically imports the agent class by reading the 04-4 notebook (C-02 cell)
#  4) Defines evaluation functions if missing: prepare_evaluation_samples, execute_evaluation,
#     analyze_evaluation_results, generate_evaluation_report
#  5) Runs a minimal evaluation and writes results under artifacts/{RUN_ID}/results
import os, json, pickle, glob, io, re, time, math, random, datetime as _dt
from typing import Dict, Any, Optional, List, Tuple
import pandas as pd

# ---------- RUN_ID resolution ----------
def _detect_run_id() -> str:
    rid = os.getenv("RUN_ID")
    if rid:
        return rid
    # find newest artifacts/* with 04-4 or 04-3 handoff
    art_root = "/mnt/data/artifacts"
    if not os.path.isdir(art_root):
        return _dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    cands = []
    for d in glob.glob(os.path.join(art_root, "*")):
        if not os.path.isdir(d): continue
        hdir = os.path.join(d, "handoff")
        if not os.path.isdir(hdir): continue
        has_04_4 = os.path.exists(os.path.join(hdir, "04-4_agent_implementation.pkl"))
        has_04_3 = os.path.exists(os.path.join(hdir, "04-3_llm_tools_setup_with_tools.pkl"))
        if has_04_4 or has_04_3:
            cands.append((os.path.getmtime(d), os.path.basename(d)))
    if not cands:
        return _dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    cands.sort(reverse=True)
    return cands[0][1]

RUN_ID = _detect_run_id()
print(f"[C-04] Using RUN_ID: {RUN_ID}")

# ---------- handoff loaders ----------
def load_04_4_handoff(run_id: Optional[str]=None) -> Dict[str, Any]:
    run_id = run_id or RUN_ID
    p = os.path.join("/mnt/data/artifacts", run_id, "handoff", "04-4_agent_implementation.pkl")
    if not os.path.exists(p):
        return {}
    with open(p, "rb") as f:
        return pickle.load(f) or {}

def load_04_3_setup(run_id: Optional[str]=None) -> Dict[str, Any]:
    run_id = run_id or RUN_ID
    p = os.path.join("/mnt/data/artifacts", run_id, "handoff", "04-3_llm_tools_setup_with_tools.pkl")
    if not os.path.exists(p):
        return {}
    with open(p, "rb") as f:
        return pickle.load(f) or {}

H4 = load_04_4_handoff(RUN_ID)
H3 = load_04_3_setup(RUN_ID)
print("[C-04] 04-4 handoff keys:", sorted(H4.keys()))
print("[C-04] 04-3 setup keys:", sorted(H3.keys())[:12], "...")

# ---------- dynamically import agent from 04-4 notebook (C-02 cell) ----------
def _import_agent_from_04_4():
    # try the patched filename (latest)
    cand = "/mnt/data/PATCHED_04-4_agent_implementation_WITH_CELL6_1761400078.ipynb"
    if not os.path.exists(cand):
        # fallback to any 04-4 patched file
        cands = sorted(glob.glob("/mnt/data/PATCHED_04-4_*.ipynb")) or sorted(glob.glob("/mnt/data/*04-4*agent*.ipynb"))
        cand = cands[-1] if cands else None
    if not cand:
        print("[C-04] 04-4 notebook not found; cannot import agent")
        return False

    with open(cand, "r", encoding="utf-8") as f:
        nb_04_4 = json.load(f)

    # find C-02 cell or a cell that defines FixedPhishingDetectionAgent
    code = None
    for cell in nb_04_4.get("cells", []):
        if cell.get("cell_type") != "code":
            continue
        meta = cell.get("metadata", {})
        tags = meta.get("tags", [])
        src = "".join(cell.get("source", ""))
        if "C-02" in tags or "FixedPhishingDetectionAgent" in src:
            code = src
            break
    if not code:
        print("[C-04] C-02 cell not found in 04-4; cannot import agent")
        return False

    # execute code in this kernel (defines make_fixed_agent, class, etc.)
    g = globals()
    try:
        exec(code, g, g)
        return True
    except Exception as e:
        print("[C-04] exec agent code failed:", e)
        return False

AGENT_OK = _import_agent_from_04_4()
if not AGENT_OK:
    raise RuntimeError("Agent import failed. Please ensure 04-4 notebook is present.")

# build agent instance
agent = make_fixed_agent(run_id=RUN_ID, trace_enabled=True)
print("[C-04] Agent ready:", type(agent).__name__)

# ---------- Evaluation functions (define only if missing) ----------
if "prepare_evaluation_samples" not in globals():
    def prepare_evaluation_samples(false_negatives_df: Optional[pd.DataFrame]=None, dev_mode: bool=True) -> pd.DataFrame:
        """Create a small evaluation sample set when original logic is unavailable."""
        random.seed(42)
        brands = H3.get("brand_keywords") or ["paypal","mercari","ledger","apple","amazon"]
        cands = [
            f"{b}-secure-login.info" for b in brands[:3]
        ] + [
            "example.com", "support-update-center.net", "my-verify-account.co", "login-validate-user.app"
        ]
        probs = [round(random.uniform(0.05, 0.9), 3) for _ in cands]
        return pd.DataFrame({"domain": cands, "ml_probability": probs})

if "execute_evaluation" not in globals():
    def execute_evaluation(sample_size: Optional[int]=None, max_concurrent: Optional[int]=None):
        df = prepare_evaluation_samples()
        if sample_size is not None:
            df = df.head(sample_size)
        rows = []
        for _, r in df.iterrows():
            res = agent.analyze(r["domain"], float(r["ml_probability"]), trace={})
            rows.append({
                "domain": r["domain"],
                "ml_probability": float(r["ml_probability"]),
                "is_phishing": bool(res.get("is_phishing", False)),
                "confidence": float(res.get("confidence", 0.0)),
                "reasoning": str(res.get("reasoning",""))[:400],
            })
        res_df = pd.DataFrame(rows)
        # write artifacts
        out_dir = os.path.join("/mnt/data/artifacts", RUN_ID, "results")
        os.makedirs(out_dir, exist_ok=True)
        csv_p = os.path.join(out_dir, "04-5_results.csv")
        res_df.to_csv(csv_p, index=False)
        return res_df, {"result_path": csv_p, "rows": len(res_df)}

if "analyze_evaluation_results" not in globals():
    def analyze_evaluation_results(results_df: pd.DataFrame) -> Dict[str, Any]:
        n = len(results_df)
        if n == 0: return {"n": 0}
        ph = int(results_df["is_phishing"].sum())
        mean_conf = float(results_df["confidence"].mean())
        return {"n": n, "predicted_phishing": ph, "rate": ph/n, "mean_confidence": mean_conf}

if "generate_evaluation_report" not in globals():
    def generate_evaluation_report(analysis: Dict[str, Any], results_df: pd.DataFrame) -> str:
        lines = []
        lines.append(f"Evaluation samples: {analysis.get('n', 0)}")
        lines.append(f"Predicted phishing: {analysis.get('predicted_phishing', 0)} (rate={analysis.get('rate',0):.2f})")
        lines.append(f"Mean confidence: {analysis.get('mean_confidence', 0.0):.3f}")
        return "\n".join(lines)

# ---------- run minimal evaluation ----------
results_df, meta = execute_evaluation(sample_size=None, max_concurrent=None)
analysis = analyze_evaluation_results(results_df)
report = generate_evaluation_report(analysis, results_df)

# save artifacts
out_dir = os.path.join("/mnt/data/artifacts", RUN_ID, "results")
os.makedirs(out_dir, exist_ok=True)
with open(os.path.join(out_dir, "04-5_analysis.json"), "w", encoding="utf-8") as f:
    json.dump(analysis, f, ensure_ascii=False, indent=2)
with open(os.path.join(out_dir, "04-5_report.txt"), "w", encoding="utf-8") as f:
    f.write(report)

print("[C-04] Evaluation completed.")
print("[C-04] Results:", meta["result_path"])
print("[C-04] Analysis:", analysis)
print("[C-04] Report path:", os.path.join(out_dir, "04-5_report.txt"))

[C-04] Using RUN_ID: 2025-10-22_074213
[C-04] 04-4 handoff keys: []
[C-04] 04-3 setup keys: [] ...
[C-04] 04-4 notebook not found; cannot import agent


RuntimeError: Agent import failed. Please ensure 04-4 notebook is present.

In [ ]:
# === C-04b: override prepare_evaluation_samples (robust) ===
import os, json, pickle, glob, warnings
import pandas as pd
from pathlib import Path

def _detect_run_id_for_045():
    rid = globals().get("RUN_ID") or os.environ.get("RUN_ID")
    if rid:
        return rid
    art_root = Path("/mnt/data/artifacts")
    if not art_root.exists():
        return None
    cands = sorted([d for d in art_root.iterdir() if d.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
    return cands[0].name if cands else None

def _load_pkl_safe(p):
    try:
        if p.exists() and p.stat().st_size > 0:
            with open(p, "rb") as f:
                return pickle.load(f) or {}
    except Exception as e:
        warnings.warn(f"[04-5] pickle load failed: {p} -> {e}")
    return {}

def _pick_prob_col(df):
    for c in ["ml_probability","ml_prob","prob","proba","pred_prob","pred_proba","probability","phish_prob","p_phish","score"]:
        if c in getattr(df, "columns", []):
            return c
    return None

def _to_eval_df(df):
    if isinstance(df, pd.DataFrame) and "domain" in df.columns:
        c = _pick_prob_col(df)
        if c:
            out = df[["domain", c]].rename(columns={c: "ml_probability"}).copy()
            return out.reset_index(drop=True)
    return None

def prepare_evaluation_samples(run_id, h44, h43, dev_mode=True):
    """Override: robustly find evaluation samples from 04-4/04-3 handoffs or CSV path."""
    tried = []

    # 1) 04-4 exports['eval_samples']
    if isinstance(h44, dict) and "eval_samples" in h44:
        df = _to_eval_df(h44["eval_samples"]) if not isinstance(h44["eval_samples"], pd.DataFrame) else h44["eval_samples"]
        if df is None and isinstance(h44["eval_samples"], list):
            try:
                df = _to_eval_df(pd.DataFrame(h44["eval_samples"]))
            except Exception:
                df = None
        if isinstance(df, pd.DataFrame) and len(df):
            print(f"[04-5] samples from 04-4 exports['eval_samples'] -> {len(df)} rows")
            return df

    # 2) 04-3 'fn_features_df'
    if isinstance(h43, dict) and "fn_features_df" in h43:
        df = _to_eval_df(h43["fn_features_df"]) if not isinstance(h43["fn_features_df"], pd.DataFrame) else h43["fn_features_df"]
        if isinstance(df, pd.DataFrame) and len(df):
            print(f"[04-5] samples from 04-3 fn_features_df -> {len(df)} rows")
            return df
        tried.append("h43['fn_features_df']")

    # 3) 04-4 meta.paths.evaluation_candidates_csv
    try:
        meta_paths = (h44 or {}).get("meta", {}).get("paths", {})
        cand = meta_paths.get("evaluation_candidates_csv")
        if cand and Path(cand).exists():
            df = pd.read_csv(cand)
            df = _to_eval_df(df) or df
            if isinstance(df, pd.DataFrame) and len(df):
                print(f"[04-5] samples loaded from {cand} -> {len(df)} rows")
                return df
        else:
            tried.append("meta.paths.evaluation_candidates_csv")
    except Exception as e:
        warnings.warn(f"[04-5] load candidates_csv failed: {e}")

    # 4) disk fallback: read current RUN_ID handoffs directly
    rid = run_id or _detect_run_id_for_045()
    if rid:
        hdir = Path("/mnt/data/artifacts") / rid / "handoff"
        h44p = hdir / "04-4_agent_implementation.pkl"
        h43p = hdir / "04-3_llm_tools_setup_with_tools.pkl"
        dd44 = _load_pkl_safe(h44p)
        dd43 = _load_pkl_safe(h43p)
        if isinstance(dd44.get("eval_samples"), pd.DataFrame):
            df = _to_eval_df(dd44["eval_samples"]) or dd44["eval_samples"]
            if isinstance(df, pd.DataFrame) and len(df):
                print(f"[04-5] samples from disk 04-4 eval_samples -> {len(df)} rows")
                return df
        if "fn_features_df" in dd43:
            df = _to_eval_df(dd43["fn_features_df"]) or dd43["fn_features_df"]
            if isinstance(df, pd.DataFrame) and len(df):
                print(f"[04-5] samples from disk 04-3 fn_features_df -> {len(df)} rows")
                return df
        tried.append(f"disk({rid})")

    # 5) Dev-mode synthetic (only when allowed)
    if dev_mode:
        rows = [
            {"domain":"paypal-login.example","ml_probability":0.91},
            {"domain":"docs-google-secure-auth.example","ml_probability":0.83},
            {"domain":"support-update-center.example","ml_probability":0.77},
            {"domain":"appleid-verify-auth.example","ml_probability":0.66},
            {"domain":"nhk.jp","ml_probability":0.05},
        ]
        print("[04-5] dev_mode=True -> using synthetic samples (5 rows)")
        return pd.DataFrame(rows)

    raise RuntimeError("[04-5] No evaluation samples found. Tried: " + "; ".join(tried[:10]))

print("[C-04b] prepare_evaluation_samples override is ready.")